In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

# Load YOLO model
yolo_model = tf.saved_model.load('yolov4-tiny-416')

# Function to preprocess image for YOLO
def preprocess_image_yolo(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, (416, 416))
    image = image / 255.0
    image = tf.expand_dims(image, 0)
    return image

# Function to draw bounding boxes on image
def draw_outputs(img, outputs, class_names):
    boxes, objectness, classes, nums = outputs
    wh = np.flip(img.shape[0:2])
    img_boxes = img.copy()
    for i in range(nums[0]):
        x1y1 = tuple((np.array(boxes[0][i][0:2]) * wh).astype(np.int32))
        x2y2 = tuple((np.array(boxes[0][i][2:4]) * wh).astype(np.int32))
        img_boxes = cv2.rectangle(img_boxes, x1y1, x2y2, (0, 255, 0), 2)
        img_boxes = cv2.putText(img_boxes, '{} {:.4f}'.format(
            class_names[int(classes[0][i])], objectness[0][i]),
            x1y1, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return img_boxes

# Load and preprocess image
image_path = tf.keras.utils.get_file(
    'YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
image = preprocess_image_yolo(image_path)

# Perform object detection
yolo_outputs = yolo_model(image)
class_names = [c.strip() for c in open("coco.names").readlines()]
image_raw = tf.io.read_file(image_path)
image_raw = tf.image.decode_image(image_raw, channels=3).numpy()

# Draw bounding boxes on the original image
img_with_boxes = draw_outputs(image_raw, yolo_outputs, class_names)

# Display the image
plt.imshow(img_with_boxes)
plt.title('YOLO Object Detection')
plt.show()

# Adversarial pattern generation function for YOLO
def create_pgd_adversarial_pattern_yolo(input_image, epsilon=0.01, alpha=0.001, iterations=40):
    adversarial_image = tf.identity(input_image)
    for _ in range(iterations):
        with tf.GradientTape() as tape:
            tape.watch(adversarial_image)
            outputs = yolo_model(adversarial_image)
            loss = tf.reduce_sum(outputs[1])  # Sum of objectness scores as loss

        gradient = tape.gradient(loss, adversarial_image)
        signed_grad = tf.sign(gradient)
        adversarial_image = adversarial_image + alpha * signed_grad
        adversarial_image = tf.clip_by_value(adversarial_image, input_image - epsilon, input_image + epsilon)
        adversarial_image = tf.clip_by_value(adversarial_image, 0, 1)  # Ensure the image stays within valid range
    return adversarial_image - input_image

# Generate adversarial pattern
perturbations = create_pgd_adversarial_pattern_yolo(image)

# Display adversarial image with epsilon = 0.01
epsilon = 0.01
adv_x = image + epsilon * perturbations
adv_x = tf.clip_by_value(adv_x, 0, 1)
adv_img_with_boxes = draw_outputs((adv_x[0] * 255).numpy().astype(np.uint8), yolo_model(adv_x), class_names)

# Display the adversarial image
plt.imshow(adv_img_with_boxes)
plt.title(f'YOLO Adversarial Image, Epsilon = {epsilon}')
plt.show()

OSError: SavedModel file does not exist at: yolov4-tiny-416/{saved_model.pbtxt|saved_model.pb}